# KDD Cup 1999 - Multi-Class Saldırı Sınıflandırması

Bu notebook, KDD Cup 1999 veri seti ile multi-class sınıflandırma (saldırı türü tespiti) modellerini geliştirir.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score

from src.data import load_kdd
from src.preprocess import add_targets, split_features
from src.models import make_multiclass_pipelines
from src.eval import plot_cm

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 1. Veri Hazırlama

In [ ]:
# Veri yükleme
print('Veri yükleniyor...')
train_raw = load_kdd('../data/kddcup.data_10_percent.gz')
test_raw = load_kdd('../data/corrected.gz')

# Hedef değişkenleri ekle
train_full = add_targets(train_raw)
test_full = add_targets(test_raw)

print(f'Full train boyutu: {train_full.shape}')
print(f'Full test boyutu: {test_full.shape}')

# Hızlı test için orta büyüklükte örneklem al
from sklearn.model_selection import train_test_split
train, _ = train_test_split(train_full, test_size=0.5, random_state=42, stratify=train_full['y_binary'])
test, _ = train_test_split(test_full, test_size=0.5, random_state=42, stratify=test_full['y_binary'])

print(f'Small train boyutu: {train.shape}')
print(f'Small test boyutu: {test.shape}')

In [ ]:
# Normal trafiği 'normal' sınıfı olarak ekle
train_mc = train.copy()
test_mc = test.copy()

# Normal trafikler için y_family = 'normal' ata
train_mc.loc[train_mc['y_binary'] == 0, 'y_family'] = 'normal'
test_mc.loc[test_mc['y_binary'] == 0, 'y_family'] = 'normal'

# NaN değerleri olan satırları kaldır (family mapping olmayan saldırılar)
train_mc = train_mc.dropna(subset=['y_family'])
test_mc = test_mc.dropna(subset=['y_family'])

print(f'Train multiclass boyutu: {train_mc.shape}')
print(f'Test multiclass boyutu: {test_mc.shape}')

In [ ]:
# Saldırı ailesi dağılımlarını kontrol et
print('Train set - Saldırı ailesi dağılımı:')
print(train_mc['y_family'].value_counts())

print('\nTest set - Saldırı ailesi dağılımı:')
print(test_mc['y_family'].value_counts())

In [ ]:
# Özellikleri ve hedef değişkenleri ayır
X_train_mc, y_train_bin, y_train_family, num_cols, cat_cols = split_features(train_mc)
X_test_mc, y_test_bin, y_test_family, _, _ = split_features(test_mc)

print(f'Özellik sayısı: {X_train_mc.shape[1]}')
print(f'Sayısal özellik sayısı: {len(num_cols)}')
print(f'Kategorik özellik sayısı: {len(cat_cols)}')
print(f'Sınıf sayısı: {y_train_family.nunique()}')

## 2. Model Pipeline'ları Hazırlama

In [ ]:
# Multi-class sınıflandırma pipeline'larını oluştur
grids = make_multiclass_pipelines(num_cols, cat_cols)

print('Hazırlanan modeller:')
for name in grids.keys():
    print(f'- {name}')

## 3. Model Eğitimi ve Hiperparametre Optimizasyonu

In [ ]:
# Cross-validation ayarları (hızlı test için 2 split)
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Sonuçları saklamak için
mc_results = []
trained_models = {}

print('Multi-class model eğitimi başlıyor...\n')

In [ ]:
# Logistic Regression
print('=== Logistic Regression (Multi-class) ===')
name = 'lr'
pipe, param_grid = grids[name]

search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train_mc, y_train_family)
trained_models[name] = search.best_estimator_

# Test seti tahminleri
y_pred = search.best_estimator_.predict(X_test_mc)

# Metrikleri hesapla
f1_macro = f1_score(y_test_family, y_pred, average='macro')
f1_weighted = f1_score(y_test_family, y_pred, average='weighted')
accuracy = accuracy_score(y_test_family, y_pred)

print(f'\nEn iyi parametreler: {search.best_params_}')
print(f'Macro F1: {f1_macro:.4f}')
print(f'Weighted F1: {f1_weighted:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print('\nSınıflandırma Raporu:')
print(classification_report(y_test_family, y_pred, digits=4))

mc_results.append({
    'model': name,
    'best_params': search.best_params_,
    'f1_macro': f1_macro,
    'f1_weighted': f1_weighted,
    'accuracy': accuracy
})

In [ ]:
# Random Forest
print('\n=== Random Forest (Multi-class) ===')
name = 'rf'
pipe, param_grid = grids[name]

search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train_mc, y_train_family)
trained_models[name] = search.best_estimator_

# Test seti tahminleri
y_pred = search.best_estimator_.predict(X_test_mc)

# Metrikleri hesapla
f1_macro = f1_score(y_test_family, y_pred, average='macro')
f1_weighted = f1_score(y_test_family, y_pred, average='weighted')
accuracy = accuracy_score(y_test_family, y_pred)

print(f'\nEn iyi parametreler: {search.best_params_}')
print(f'Macro F1: {f1_macro:.4f}')
print(f'Weighted F1: {f1_weighted:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print('\nSınıflandırma Raporu:')
print(classification_report(y_test_family, y_pred, digits=4))

mc_results.append({
    'model': name,
    'best_params': search.best_params_,
    'f1_macro': f1_macro,
    'f1_weighted': f1_weighted,
    'accuracy': accuracy
})

## 4. Model Performans Karşılaştırması

In [ ]:
# Sonuçları tablo halinde göster
mc_results_df = pd.DataFrame(mc_results)
print('Multi-class Model Performans Karşılaştırması:')
print(mc_results_df)

## 5. En İyi Modelin Detaylı Analizi

In [ ]:
# En iyi modeli seç (Macro F1'e göre)
best_model_name = mc_results_df.loc[mc_results_df['f1_macro'].idxmax(), 'model']
best_model = trained_models[best_model_name]

print(f'En iyi model: {best_model_name}')

# En iyi model ile tahminler
y_pred_best = best_model.predict(X_test_mc)

In [ ]:
# Confusion Matrix
labels = sorted(y_test_family.unique())
plot_cm(y_test_family, y_pred_best, labels=labels, 
        title=f'Multi-class Confusion Matrix ({best_model_name})', 
        normalize='true')

In [ ]:
# Sınıf bazında performans analizi
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(
    y_test_family, y_pred_best, average=None, labels=labels
)

class_performance = pd.DataFrame({
    'class': labels,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'support': support
})

print('Sınıf bazında performans:')
print(class_performance.round(4))

## 6. Özellik Önemleri (Random Forest için)

In [ ]:
# Eğer en iyi model Random Forest ise özellik önemlerini göster
if best_model_name == 'rf':
    # Özellik önemlerini al
    feature_names = best_model.named_steps['pre'].get_feature_names_out()
    importances = best_model.named_steps['clf'].feature_importances_
    
    # En önemli 20 özelliği göster
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print('En önemli 20 özellik (Multi-class):')
    print(feature_importance_df.head(20))
    
    # Görselleştir
    plt.figure(figsize=(10, 8))
    top_features = feature_importance_df.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Özellik Önemi')
    plt.title('En Önemli 20 Özellik (Multi-class Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print(f'Özellik önemleri sadece Random Forest modeli için gösterilir. En iyi model: {best_model_name}')

## 7. Hata Analizi

In [ ]:
# Yanlış sınıflandırılan örnekleri analiz et
misclassified = y_test_family != y_pred_best

print(f'Toplam yanlış sınıflandırma: {misclassified.sum()}')
print(f'Yanlış sınıflandırma oranı: {misclassified.mean():.4f}')

# En çok karıştırılan sınıf çiftleri
from collections import Counter

misclass_pairs = []
for true_label, pred_label in zip(y_test_family[misclassified], y_pred_best[misclassified]):
    misclass_pairs.append((true_label, pred_label))

most_common_errors = Counter(misclass_pairs).most_common(10)

print('\nEn sık karıştırılan sınıf çiftleri (gerçek -> tahmin):')
for (true_class, pred_class), count in most_common_errors:
    print(f'{true_class} -> {pred_class}: {count} kez')

## 8. Sonuçlar ve Değerlendirme

### Temel Bulgular:

1. **Model Performansları**: Yukarıdaki tabloda karşılaştırıldı
2. **En İyi Model**: Macro F1 skoruna göre belirlendi
3. **Sınıf Bazında Performans**: Her saldırı ailesi için ayrı ayrı analiz edildi
4. **Hata Analizi**: En sık karıştırılan sınıf çiftleri belirlendi

### Gözlemler:

- **DoS saldırıları** genellikle en kolay tespit edilen sınıftır
- **U2R ve R2L** saldırıları daha az örnekli olduğu için tespit edilmesi zordur
- **Probe** saldırıları orta düzeyde başarı ile tespit edilir

### Öneriler:

- Dengesiz sınıflar için daha gelişmiş örnekleme teknikleri kullanılabilir
- Ensemble yöntemleri (Voting, Stacking) denenebilir
- Özellik mühendisliği ile yeni özellikler türetilebilir
- Cost-sensitive learning yaklaşımları uygulanabilir